In [ ]:
import numpy as np
import pandas as pd
import rasterio as rs


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:


img0 = rs.open('/content/drive/MyDrive/BTP/a_0.tif')
img1 = rs.open('/content/drive/MyDrive/BTP/a_1.tif')
img2 = rs.open('/content/drive/MyDrive/BTP/a_2.tif')
img3 = rs.open('/content/drive/MyDrive/BTP/a_3.tif')

In [ ]:
img3.shape

(4296, 9609)

In [ ]:
img0.meta

{'driver': 'GTiff',
 'dtype': 'float64',
 'nodata': None,
 'width': 9609,
 'height': 4296,
 'count': 1,
 'crs': CRS.from_epsg(4326),
 'transform': Affine(8.983152841195215e-05, 0.0, 85.82407708642353,
        0.0, -8.983152841195215e-05, 25.57619883224871)}

In [ ]:
data=pd.read_csv('/content/drive/MyDrive/BTP/BTP_a_0.csv')

In [ ]:
data

,CLASS,Longitude,Latitude
0,2,86.519368,25.192734
1,2,86.543441,25.194933
2,2,86.499428,25.194989
3,1,86.170355,25.204459
4,1,86.175672,25.213510
...,...,...,...
240,1,86.521517,25.505425
241,1,86.509637,25.505931
242,1,86.285749,25.507130
243,1,86.294811,25.515322


In [ ]:
transform = rs.Affine(8.983152841195215e-05, 0.0, 85.82407708642353,
        0.0, -8.983152841195215e-05, 25.57619883224871)
transformer = rs.transform.AffineTransformer(transform)

In [ ]:
def add_xy_coordinates_and_classification(water):
    x = []
    y = []
    cl= []
    k = water.shape[0]
    for i in range(k):
        lat = water.loc[i].at["Latitude"]
        long = water.loc[i].at["Longitude"]
        clf = water.loc[i].at["CLASS"]
        a = transformer.rowcol(long, lat)
        x.append(int(a[0])) 
        y.append(int(a[1]))
        cl.append(int(clf))
    water["xc"] = x
    water["yc"] = y
    water["Classification"] = cl
    return water
    

In [ ]:
add_xy_coordinates_and_classification(data)

,CLASS,Longitude,Latitude,xc,yc,Classification
0,2,86.519368,25.192734,4268,7739,2
1,2,86.543441,25.194933,4244,8007,2
2,2,86.499428,25.194989,4243,7517,2
3,1,86.170355,25.204459,4138,3854,1
4,1,86.175672,25.213510,4037,3913,1
...,...,...,...,...,...,...
240,1,86.521517,25.505425,787,7763,1
241,1,86.509637,25.505931,782,7631,1
242,1,86.285749,25.507130,768,5139,1
243,1,86.294811,25.515322,677,5240,1


In [ ]:
# taking band values as numpy arrays
x0 = img0.read(1).astype(float)
x1 = img1.read(1).astype(float)
x2 = img2.read(1).astype(float)
x3 = img3.read(1).astype(float)

bands = [x0,x1,x2,x3]
X = []

for _ in range(len(bands)):
    X.append([])
k = data.shape[0]
for i in range(k):
    x = int(data.loc[i].at["xc"])
    y = int(data.loc[i].at["yc"])
    for j in range(len(bands)):
        X[j].append(bands[j][x, y]);
name = "Band "
cnt = 1
for i in range(len(X)):
    col_name = name+str(cnt)
    data[col_name] = X[i]
    cnt = cnt + 1
data

,CLASS,Longitude,Latitude,xc,yc,Classification,Band 1,Band 2,Band 3,Band 4
0,2,86.519368,25.192734,4268,7739,2,0.081659,0.014067,0.005508,0.006385
1,2,86.543441,25.194933,4244,8007,2,0.050733,0.003595,0.002719,0.003094
2,2,86.499428,25.194989,4243,7517,2,0.037993,0.001440,0.004863,0.008960
3,1,86.170355,25.204459,4138,3854,1,0.006182,0.001865,0.001548,0.001055
4,1,86.175672,25.213510,4037,3913,1,0.004793,0.000784,0.000051,0.000064
...,...,...,...,...,...,...,...,...,...,...
240,1,86.521517,25.505425,787,7763,1,0.005246,0.000381,0.000698,0.000235
241,1,86.509637,25.505931,782,7631,1,0.008409,0.002107,0.001634,0.000870
242,1,86.285749,25.507130,768,5139,1,0.004117,0.000167,0.000322,0.000292
243,1,86.294811,25.515322,677,5240,1,0.003673,0.000127,0.000517,0.000697


In [ ]:
data.drop(["xc", "yc","Longitude","Latitude", "Classification" ],axis=1,inplace=True)

In [ ]:
data

,CLASS,Band 1,Band 2,Band 3,Band 4
0,2,0.081659,0.014067,0.005508,0.006385
1,2,0.050733,0.003595,0.002719,0.003094
2,2,0.037993,0.001440,0.004863,0.008960
3,1,0.006182,0.001865,0.001548,0.001055
4,1,0.004793,0.000784,0.000051,0.000064
...,...,...,...,...,...
240,1,0.005246,0.000381,0.000698,0.000235
241,1,0.008409,0.002107,0.001634,0.000870
242,1,0.004117,0.000167,0.000322,0.000292
243,1,0.003673,0.000127,0.000517,0.000697


In [ ]:
X = data.iloc[:, 1:5]
X

,Band 1,Band 2,Band 3,Band 4
0,0.081659,0.014067,0.005508,0.006385
1,0.050733,0.003595,0.002719,0.003094
2,0.037993,0.001440,0.004863,0.008960
3,0.006182,0.001865,0.001548,0.001055
4,0.004793,0.000784,0.000051,0.000064
...,...,...,...,...
240,0.005246,0.000381,0.000698,0.000235
241,0.008409,0.002107,0.001634,0.000870
242,0.004117,0.000167,0.000322,0.000292
243,0.003673,0.000127,0.000517,0.000697


In [ ]:
y = data.iloc[:,0]
y

0      2
1      2
2      2
3      1
4      1
      ..
240    1
241    1
242    1
243    1
244    1
Name: CLASS, Length: 245, dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=5)



In [ ]:
import xgboost as xgb
from xgboost import XGBClassifier

model = XGBClassifier()
model.get_params()



{'base_score': 0.5,
 'booster': 'gbtree',
 'colsample_bylevel': 1,
 'colsample_bynode': 1,
 'colsample_bytree': 1,
 'gamma': 0,
 'learning_rate': 0.1,
 'max_delta_step': 0,
 'max_depth': 3,
 'min_child_weight': 1,
 'missing': None,
 'n_estimators': 100,
 'n_jobs': 1,
 'nthread': None,
 'objective': 'binary:logistic',
 'random_state': 0,
 'reg_alpha': 0,
 'reg_lambda': 1,
 'scale_pos_weight': 1,
 'seed': None,
 'silent': None,
 'subsample': 1,
 'verbosity': 1}

In [ ]:
model.fit(X_train, y_train)

XGBClassifier(objective='multi:softprob')

In [ ]:
y_pred = model.predict(X_test) 

In [ ]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test,y_pred) 
print("Accuracy: %.2f%%" % (accuracy * 100.0))



Accuracy: 65.31%


In [ ]:
from sklearn.model_selection import RandomizedSearchCV
import xgboost
classifier = xgboost.XGBClassifier()
params={
 "learning_rate"    : [0.02,0.04,0.06,0.08,0.10,0.12,0.14,0.16,0.18,0.20,0.22,0.24,0.26,0.28,0.30 ] ,
 "max_depth"        : [ 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
 "min_child_weight" : [ 1, 3, 5, 7 ],
 "gamma"            : [ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
 "colsample_bytree" : [ 0.3, 0.4, 0.5 , 0.7 ]
    
}

In [ ]:
rs_model=RandomizedSearchCV(classifier,param_distributions=params,n_iter=5,scoring='roc_auc',n_jobs=-1,cv=5,verbose=3)

In [ ]:
rs_model.fit(X,y)


Fitting 5 folds for each of 5 candidates, totalling 25 fits


/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_search.py:969: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan]
  warnings.warn(


RandomizedSearchCV(cv=5, estimator=XGBClassifier(), n_iter=5, n_jobs=-1,
                   param_distributions={'colsample_bytree': [0.3, 0.4, 0.5,
                                                             0.7],
                                        'gamma': [0.0, 0.1, 0.2, 0.3, 0.4],
                                        'learning_rate': [0.02, 0.04, 0.06,
                                                          0.08, 0.1, 0.12, 0.14,
                                                          0.16, 0.18, 0.2, 0.22,
                                                          0.24, 0.26, 0.28,
                                                          0.3],
                                        'max_depth': [3, 4, 5, 6, 7, 8, 9, 10,
                                                      11, 12, 13, 14, 15],
                                        'min_child_weight': [1, 3, 5, 7]},
                   scoring='roc_auc', verbose=3)

In [ ]:
rs_model.best_estimator_


XGBClassifier(colsample_bytree=0.5, gamma=0.1, learning_rate=0.18, max_depth=5,
              objective='multi:softprob')

In [ ]:
rs_model.best_params_

{'min_child_weight': 1,
 'max_depth': 5,
 'learning_rate': 0.18,
 'gamma': 0.1,
 'colsample_bytree': 0.5}

In [ ]:
from sklearn.model_selection import cross_val_score
score=cross_val_score(classifier,X,y,cv=10)

In [ ]:
print("Accuracy: %.2f%%" % (score.mean()*100))

Accuracy: 67.30%


In [ ]:
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(classifier, params, verbose=2)
grid.fit(X_train, y_train)


Streaming output truncated to the last 5000 lines.
[CV] END colsample_bytree=0.7, gamma=0.3, learning_rate=0.22, max_depth=13, min_child_weight=1; total time=   0.1s
[CV] END colsample_bytree=0.7, gamma=0.3, learning_rate=0.22, max_depth=13, min_child_weight=1; total time=   0.1s
[CV] END colsample_bytree=0.7, gamma=0.3, learning_rate=0.22, max_depth=13, min_child_weight=1; total time=   0.1s
[CV] END colsample_bytree=0.7, gamma=0.3, learning_rate=0.22, max_depth=13, min_child_weight=1; total time=   0.1s
[CV] END colsample_bytree=0.7, gamma=0.3, learning_rate=0.22, max_depth=13, min_child_weight=1; total time=   0.1s
[CV] END colsample_bytree=0.7, gamma=0.3, learning_rate=0.22, max_depth=13, min_child_weight=3; total time=   0.0s
[CV] END colsample_bytree=0.7, gamma=0.3, learning_rate=0.22, max_depth=13, min_child_weight=3; total time=   0.0s
[CV] END colsample_bytree=0.7, gamma=0.3, learning_rate=0.22, max_depth=13, min_child_weight=3; total time=   0.1s
[CV] END colsample_bytree=0.7

GridSearchCV(estimator=XGBClassifier(),
             param_grid={'colsample_bytree': [0.3, 0.4, 0.5, 0.7],
                         'gamma': [0.0, 0.1, 0.2, 0.3, 0.4],
                         'learning_rate': [0.02, 0.04, 0.06, 0.08, 0.1, 0.12,
                                           0.14, 0.16, 0.18, 0.2, 0.22, 0.24,
                                           0.26, 0.28, 0.3],
                         'max_depth': [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14,
                                       15],
                         'min_child_weight': [1, 3, 5, 7]},
             verbose=2)

In [ ]:
grid.best_params_

{'colsample_bytree': 0.5,
 'gamma': 0.1,
 'learning_rate': 0.26,
 'max_depth': 8,
 'min_child_weight': 3}

In [ ]:
print(grid.best_score_*100,"%")

69.37179487179486 %


In [ ]:
x0 = img0.read(1).astype(float)
x1 = img1.read(1).astype(float)
x2 = img2.read(1).astype(float)
x3 = img3.read(1).astype(float)

bands = [x0,x1,x2,x3]

In [ ]:
clf_data = pd.DataFrame(columns=['xc','yc'])
clf_data
p =[]
q =[]
for i in range(x1.shape[0]):
    for j in range(x1.shape[1]):
        p.append(i)
        q.append(j)
clf_data["xc"]=p
clf_data["yc"]=q

In [ ]:
clf_data

,xc,yc
0,0,0
1,0,1
2,0,2
3,0,3
4,0,4
...,...,...
41280259,4295,9604
41280260,4295,9605
41280261,4295,9606
41280262,4295,9607


In [1]:
X = []
for _ in range(len(bands)):
    X.append([])
k = clf_data.shape[0]
for i in range(k):
    x = int(clf_data.loc[i].at["xc"])
    y = int(clf_data.loc[i].at["yc"])
    for j in range(len(bands)):
        X[j].append(bands[j][x, y]);
name = "Band "
cnt = 1
for i in range(len(X)):
    col_name = name+str(cnt)
    clf_data[col_name] = X[i]
    cnt = cnt + 1

NameError: ignored